In [1]:
import cv2
import os
import cvlib as cv
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np

from PyQt6.QtGui import QIcon, QFont
from PyQt6 import QtCore, QtGui, QtWidgets
import cv2

# Tải các file h5 sau khi train model
model_age = load_model('AgeOfficial.h5')
model_emotion = load_model('EmotionOfficial.h5')
model_gender = load_model('GenderOfficial.h5')

# Mở webcam
webcam = cv2.VideoCapture(0)

# Tạo list các giá trị dự đoán cho từng mô hình
classes_model_age = ['Under 12','53-57','58-62','Over 63','13-17','18-22','23-27','28-32','33-37','38-42','43-47','48-52']
classes_model_emotion = ['Neutral','Happy','Sad','Surprised','Angry']
classes_model_gender = ['Male','Female']

class Ui_Dialog(object):
    def setupUi(self, Dialog):
        Dialog.setObjectName("Nhận diện khuôn mặt")
        Dialog.resize(700, 700)

        # Chỉnh tiêu đề 1
        self.label_title = QtWidgets.QLabel(Dialog)
        self.label_title.setGeometry(QtCore.QRect(50, -250, 601, 561))
        self.label_title.setStyleSheet('color:blue')
        font = QtGui.QFont()
        font.setPointSize(12)
        font.setBold(True)
        font.setWeight(75)
        self.label_title.setFont(font)
        self.label_title.setObjectName("label_title")
        self.label_title.setAlignment(QtCore.Qt.AlignmentFlag.AlignCenter)

        # Chỉnh tiêu đề 2
        self.label_title2 = QtWidgets.QLabel(Dialog)
        self.label_title2.setGeometry(QtCore.QRect(50, -220, 601, 561))
        self.label_title2.setStyleSheet('color:blue')
        font = QtGui.QFont()
        font.setPointSize(12)
        font.setBold(True)
        font.setWeight(75)
        self.label_title2.setFont(font)
        self.label_title2.setObjectName("label_title2")
        self.label_title2.setAlignment(QtCore.Qt.AlignmentFlag.AlignCenter)

        # Chỉnh tiêu đề 3
        self.label_title3 = QtWidgets.QLabel(Dialog)
        self.label_title3.setGeometry(QtCore.QRect(50, -190, 601, 561))
        self.label_title3.setStyleSheet('color:blue')
        font = QtGui.QFont()
        font.setPointSize(12)
        font.setBold(True)
        font.setWeight(75)
        self.label_title3.setFont(font)
        self.label_title3.setObjectName("label_title3")
        self.label_title3.setAlignment(QtCore.Qt.AlignmentFlag.AlignCenter)


        # Chỉnh khung camera
        self.label = QtWidgets.QLabel(Dialog)
        self.label.setGeometry(QtCore.QRect(150, 120, 400, 400))
        self.label.setText("")
        self.label.setScaledContents(True)
        self.label.setObjectName("label")

        # Button Start 
        self.pushButton = QtWidgets.QPushButton(Dialog)
        self.pushButton.setGeometry(QtCore.QRect(100, 550, 191, 71))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.pushButton.setFont(font)
        self.pushButton.setObjectName("pushButton")

        # Button Stop
        self.pushButtonStop = QtWidgets.QPushButton(Dialog)
        self.pushButtonStop.setGeometry(QtCore.QRect(400, 550, 191, 71))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.pushButtonStop.setFont(font)
        self.pushButtonStop.setObjectName("pushButtonStop")



        self.retranslateUi(Dialog)
        QtCore.QMetaObject.connectSlotsByName(Dialog)

    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("TuanPham", "TuanPham"))

        self.pushButton.setText(_translate("Dialog", "CLICK TO START"))    
        self.pushButton.clicked.connect(self.use_camera)

        self.pushButtonStop.setText(_translate("Dialog", "CLICK TO STOP"))    
        # self.pushButtonStop.clicked.connect(self.main_win.close)

        self.label_title.setText(_translate("Dialog", "Dự đoán tuổi tác, giới tính và cảm xúc dựa vào khuôn mặt người"))
        self.label_title2.setText(_translate("Dialog", "Môn học: Trí tuệ nhân tạo; Ngày học: Thứ 6 (Tiết 12-15)"))
        self.label_title3.setText(_translate("Dialog", "SVTH: Phạm Minh Tuấn; MSSV: 19146297; GVHD: PGS.TS Nguyễn Trường Thịnh"))

    def use_camera(self, Dialog):
        self.Worker1 = Worker1()
        self.Worker1.start()
        self.Worker1.ImageUpdate.connect(self.ImageUpdateSlot)
        self.is_running = True
    
    def ImageUpdateSlot(self, Image):
        self.label.setPixmap(QtGui.QPixmap.fromImage(Image))

    def CancelFeed(self):
        self.Worker1.stop()

class Worker1(QtCore.QThread):
    ImageUpdate = QtCore.pyqtSignal(QtGui.QImage)
    def run(self):
        self.ThreadActive = True
        Capture = cv2.VideoCapture(0)
        while self.ThreadActive:
            ret, frame = Capture.read()
            
            # Bắt đầu nhận dạng khuôn mặt
            face, confidence = cv.detect_face(frame)


            for idx, f in enumerate(face):

                # Lấy các điểm theo hình chữ nhật      
                (startX, startY) = f[0], f[1]
                (endX, endY) = f[2], f[3]

                # Vẽ hình chữ nhật xung quanh khuôn mặt
                cv2.rectangle(frame, (startX,startY), (endX,endY), (255,255,0), 2)

                # Cắt vùng khuôn mặt được phát hiện
                face_crop = np.copy(frame[startY:endY,startX:endX])

                if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
                    continue
            
                # Tiền xử lý cho mô hình dự đoán
                face_crop = cv2.resize(face_crop, (150,150))
                face_crop = face_crop.astype("float") / 255.0
                face_crop = img_to_array(face_crop)
                face_crop = np.expand_dims(face_crop, axis=0)

                # Dự đoán tuổi
                conf_model_age = model_age.predict(face_crop)[0]
                # Lấy nhãn với độ chính xác cao nhất
                idx_model_age = np.argmax(conf_model_age)
                label_model_age = classes_model_age[idx_model_age-2]

                # Dự đoán giới tính
                conf_model_gender = model_gender.predict(face_crop)[0]
                # Lấy nhãn với độ chính xác cao nhất
                idx_model_gender = np.argmax(conf_model_gender)
                label_model_gender = classes_model_gender[idx_model_gender]

                # Dự đoán cảm xúc
                conf_model_emotion = model_emotion.predict(face_crop)[0]
                # Lấy nhãn với độ chính xác cảm xúc
                idx_model_emotion = np.argmax(conf_model_emotion)
                label_model_emotion = classes_model_emotion[idx_model_emotion]


                label = "{},{},{}".format(label_model_age,label_model_gender,label_model_emotion)

                Y = startY - 10 if startY - 10 > 10 else startY + 10

                # Viết nhãn các giá trị dự đoán bên trên khung hình chữ nhật
                cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (255, 255, 0), 2)
                
            # # Hiển thị đầu ra
            # cv2.imshow("detection", frame)

            # Nhấn s để dừng
            if cv2.waitKey(1) & 0xFF == ord('s'):
                break

            if ret:
                Image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                ConvertToQtFormat = QtGui.QImage(Image.data, Image.shape[1], Image.shape[0], QtGui.QImage.Format.Format_RGB888)
                Pic = ConvertToQtFormat.scaled(640, 480, QtCore.Qt.AspectRatioMode.KeepAspectRatio)
                self.ImageUpdate.emit(Pic)

        # listen to press F5 to stop thread
        if self.ThreadActive:
            self.ThreadActive = False
            Capture.release()
            cv2.destroyAllWindows()

        
    def stop(self):
        self.ThreadActive = False
        self.quit()



if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    Dialog = QtWidgets.QDialog()
    ui = Ui_Dialog()
    ui.setupUi(Dialog)
    Dialog.show()
    sys.exit(app.exec())


1/1 [==============================] - 0s 30ms/step


SystemExit: 0

c:\Users\minht\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


1/1 [==============================] - 0s 44ms/step
